Import TensorFlow and other libraries.

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import os
import time

Load the model.

In [ ]:
one_step_model_custom = tf.saved_model.load('one_step')

That's no real entries for SCP Foundation, so fake indices will be generated and used as prompts. Also, model will be referencing other SCPs and as model doesn't have a clue what are those SCPs it's better to just hide those indices. 

In [ ]:
import random
import re

def fake_index(): 
    return str(random.randint(10000, 99999))

def hidden_indices(text: str):
    indices = re.findall(r"SCP-\d*", text)
    to_replace = zip(indices, map(lambda x: 'SCP-' + '█████', indices))

    for pair in to_replace:
        text = text.replace(pair[0], pair[1])

    return text

In [ ]:
def start_prompt():
    return '☺SCP-' + fake_index() + ' \n☺"'

As constant size texts not always come out finished '\n' output considered as terminal condition.

In [ ]:
def generate_continiously(step_model, prompt):
  def not_finished(result):
    return result[-1] != '\n'
  
  states = None
  next_char = tf.constant([prompt])
  result = [next_char]

  while not_finished(result):
    next_char, states = step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  return result[0].numpy().decode('utf-8')

SCP "file" is generated by blocks, then merged.

In [ ]:
def generate_scp(step_model):
    parts = [start_prompt(), '☺Object Class: ', '☺Special Containment Procedures: ', '☺Description: ']
    generated = list(map(lambda x: generate_continiously(x), parts))

    for i in range(1, 4):
        generated[i] = hidden_indices(generated[i]) 

    return ''.join(generated)
    

Don't forget to remove ☺.

P.S. ☺

In [ ]:
scp = generate_scp(one_step_model_custom).replace('☺', '')
print(scp)